##  Understanding the Ella CLI

The Ella CLI is very robust and you can learn alot just by running subcommands and `--help`. You can manipulate any of the datamodels through the CLI.

## Bring up the Dev Stack

Bring up the dev stack and make the database schema.

```bash
# From the project root
make dev
make db
```

## Users, Groups, and FilterConfigs

At a minimum to use Ella you need users, groups, and filter configs.

### Add User Groups

Since we'll be using the demo data in Ella we need to add the User Groups first, or else adding our users will fail because it will point to user groups that don't exist.

```bash
echo "Adding user groups"
docker-compose -f local.yml exec \
		ella-web bash -c "ella-cli users add_groups /ella/src/vardb/testdata/usergroups.json"
```

### Add Users

Now that we have the user groups we can add the users.

```bash
echo "Adding users"
docker-compose -f local.yml \
		exec ella-web bash -c "ella-cli users add_many /ella/src/vardb/testdata/users.json"
```

### Add Filter Configs

Filter configs can be very complex. The idea is to use a computer to filter what the variant scientist sees on the Ella UI. To get started we'll just add in the test data filter config

```bash
echo "Adding filter configs"
docker-compose -f local.yml \
		exec ella-web bash -c "ella-cli filterconfigs update /ella/src/vardb/testdata/filterconfigs.json"
```

## Gene Panels

If you hammer on the test data you'll see that there are plenty of gene panels you can use.


```bash
echo "Create HBOC v01 gene panel"
docker-compose -f local.yml exec ella-web \
    bash -c \
    "ella-cli deposit genepanel --genepanel_name HBOC --genepanel_version v01 \
        --transcripts_path /ella/src/vardb/testdata/clinicalGenePanels/HBOC_v01/HBOC_v01.transcripts.csv \
        --phenotypes_path /ella/src/vardb/testdata/clinicalGenePanels/HBOC_v01/HBOC_v01.phenotypes.csv"

echo "Create HBOCUTV v01 gene panel"
docker-compose -f local.yml exec  ella-web \
    bash -c \
    "ella-cli deposit genepanel --genepanel_name HBOCUTV --genepanel_version v01 \
        --transcripts_path /ella/src/vardb/testdata/clinicalGenePanels/HBOCUTV_v01/HBOCUTV_v01.transcripts.csv \
        --phenotypes_path /ella/src/vardb/testdata/clinicalGenePanels/HBOCUTV_v01/HBOCUTV_v01.phenotypes.csv"

echo "Create Mendeliome v01 gene panel"
docker-compose -f local.yml exec  ella-web \
    bash -c \
    "ella-cli deposit genepanel --genepanel_name Mendeliome --genepanel_version v01 \
        --transcripts_path /ella/src/vardb/testdata/clinicalGenePanels/Mendeliome_v01/Mendeliome_v01.transcripts.csv \
        --phenotypes_path /ella/src/vardb/testdata/clinicalGenePanels/Mendeliome_v01/Mendeliome_v01.phenotypes.csv"

echo "Create Ciliopati v05 gene panel"
docker-compose -f local.yml exec ella-web \
    bash -c \
    "ella-cli deposit genepanel --genepanel_name Ciliopati --genepanel_version v05 \
        --transcripts_path /ella/src/vardb/testdata/clinicalGenePanels/Ciliopati_v05/Ciliopati_v05.transcripts.csv \
        --phenotypes_path /ella/src/vardb/testdata/clinicalGenePanels/Ciliopati_v05/Ciliopati_v05.phenotypes.csv"
```

## Analyses

Analyses are associated to a genepanel, which is associated to a User group, and the User Group is associated to a filter config.

The easiest way to load an analysis into Ella is to use the CLI and to load a VCF file that was annotated with the Ella Annotation Service. The VCF file should be named with the format `{analysis_name}.{genepanel_name}_{genepanel_version}.vcf`. If its compressed it should have a tabix index.

```bash
docker-compose -f local.yml \
	exec ella-web \
	bash -c "find /ella/src/vardb/testdata/analyses/default -name '*vcf' | grep -v Trio | grep -v brca_sample_2 | xargs -I {} ella-cli deposit analysis {}"
```

If you see an error like:

```bash
psycopg2.OperationalError: stack depth limit exceeded
HINT:  Increase the configuration parameter "max_stack_depth" (currently 2048kB), after ensuring the platform's stack depth limit is adequate.
```
 
This means that the machine you are on is not powerful enough to handle the database transaction. For the purposes of demonstration this doesn't really matter as long as you can load in at least a few analyses to look at. 

**Coming Soon** If you are planning on loading large analyses (whole exome, whole genome) I will soon have a reworked function to do that in Ella Admin.

You'll notice we reverse grep a few vcf files. This is because they are mutli sample VCF files that require a ped file.

```bash
echo "Adding trio analyses"
docker-compose -f local.yml \
	exec ella-web \
	bash -c "ella-cli deposit analysis \
		--ped /ella/src/vardb/testdata/analyses/default/HG002-Trio.Mendeliome_v01/HG002-Trio.Mendeliome_v01.ped \
		/ella/src/vardb/testdata/analyses/default/HG002-Trio.Mendeliome_v01/HG002-Trio.Mendeliome_v01.vcf"
```